In [1]:
import numpy as np
import pandas as pd 
import json
import os
from multiprocessing import Pool
import re

In [2]:
df1 = pd.DataFrame({'num':[60,20,80,90],
                    'rate':[2,1,3,4],
                    'value':[40,50,60,55]},
                   index=[2001,2002,2003,2004])
# In[25]:
df2 = pd.DataFrame({'num':[60,20,80,90],
                    'rate':[2,1,3,4],
                    'value':[40,50,60,55]},
                   index=[2005,2006,2007,2008])

In [5]:
pd.concat([df1,df2])

,num,rate,value
2001,60,2,40
2002,20,1,50
2003,80,3,60
2004,90,4,55
2005,60,2,40
2006,20,1,50
2007,80,3,60
2008,90,4,55


In [16]:
import re
pattern=re.compile('[0-9]+\t{.+"d":.+}')
s='75640178358\t{"id":"75640178358","crawler_time":"2022-10-08 00:00:00","crawler_time_stamp":"1665158400000","is_retweet":"0","user_id":"1454303565","nick_name":"希阳西","tou_xiang":"https:\\/\\/tvax2.sinaimg.cn\\/crop.0.0.512.512.50\\/56aee94dly8gjewniter9j20e80e8jsa.jpg?KID=imgbed,tva&Expires=1665169199&ssig=G7HjHOHM1q","user_type":"普通用户","weibo_id":"4822065498821328","weibo_content":"#肖战[超话]##肖战时影#[心]#肖战肖春生# \\n韶华流光里，\\n俯仰皆是你@X玖少年团肖战DAYTOY @X玖少年团肖战DAYTOY [心][兔子][贊啊]#肖战顺顺利利# \u200b\u200b\u200b","zhuan":"0","ping":"1","zhan":"1","url":"M9g1vB0Pu","device":"肖戰順順利利","locate":"","time":"2022-10-07 22:37:47","time_stamp":"1665153467","r_user_id":"","r_nick_name":"","r_user_type":"","r_weibo_id":"","r_weibo_content":"","r_zhuan":"","r_ping":"","r_zhan":"","r_url":"","r_device":"","r_location":"","r_time":"","r_time_stamp":"","pic_content":"56aee94dgy1h6x3jo6vz7j236c2jxkjl","src":"4","tag":"0","vedio":"0","vedio_image":"","edited":"0","r_edited":"","isLongText":"0","r_isLongText":"","lat":"","lon":"","ext":"{\\"region_name\\":\\"发布于 广东\\}'

In [17]:
print(pattern.search(s))

None


In [19]:
def process_chunk(chunk):
    # 将chunk中的每个字典去掉编号并返回一个DataFrame
    l=list(filter(None,chunk.split('\n')))
    return pd.DataFrame(l,columns=['wb'])

def handle_error(err):
    print("an error occured ",err)
    
def read_file_to_dataframe(filename, chunk_size=1024*1024, workers=4):
    # 获取文件大小
    file_size = os.path.getsize(filename)

    # 定义进程池和结果列表
    pool = Pool(processes=workers)
    df=pd.DataFrame(columns=['wb'])
    # 逐个chunk处理
    with open(filename, 'r', encoding='utf-8') as f:
        while f.tell() < file_size:
            # 读取一个chunk的数据
            chunk = []
            for i in range(chunk_size):
                line = f.readline()
                if not line:
                    break
                chunk.append(line)

            # 处理一个chunk的数据
            res= pool.apply_async(process_chunk, args=(chunk, ),error_callback=handle_error)
            try:
                df.append(res.get())
            except Exception as e:
                print("an err occured:{}".format(e))
        # 等待所有进程完成
        pool.close()
        pool.join()

    # 将结果存入DataFrame
    df.to_excel('ans.xlsx')
    return df